In [1]:
#############################################################################
#
# Make Labeled Reaching Movies
# 02/25/20
#
#############################################################################
import numpy as np
import os, sys
from datetime import datetime as dt
import importlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import json
import random
import re
from PIL import Image, ImageFont, ImageDraw
import cv2
import dlib
import argparse
import imutils
import time
from imutils.video import VideoStream
import statistics
from pathlib import Path
import time
from scipy.stats import t
import reaching_utils as ru
import cv_utils as cvu
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

In [2]:
#############################################################################
#
# Define data folders
#
#############################################################################
codeFolder = Path().absolute()
dDrive = os.path.dirname(codeFolder)
movieFolder = 'reachingvideos'
moviePath = os.path.join(dDrive, movieFolder, 'waterdrop', 'labeled')
dlcFolder = 'DeepLabCut'
projectFolder = 'projects'
reachingTopFolder = 'ReachingTop-Sarah-2020-02-24'
reachingBottomFolder = 'ReachingBottom-Sarah-2020-02-25'
dlcVideoFolder = 'videos'

topVideoPath = os.path.join(dDrive, dlcFolder, projectFolder, reachingTopFolder, dlcVideoFolder)
bottomVideoPath = os.path.join(dDrive, dlcFolder, projectFolder, reachingBottomFolder, dlcVideoFolder)


In [3]:
movName = '20200221_WT1_D1'
topMovName = movName + '_top'
bottomMovName = movName + '_bottom'

topMovPath = os.path.join(topVideoPath, topMovName + '.avi')
bottomMovPath = os.path.join(bottomVideoPath, bottomMovName + '.avi')

topLabelInfo = 'DeepCut_resnet50_ReachingTopFeb24shuffle1_14900'
bottomLabelInfo = 'DeepCut_resnet50_ReachingBottomFeb25shuffle1_66600'
topLabelPath = os.path.join(topVideoPath, topMovName + topLabelInfo + '.csv')
bottomLabelPath = os.path.join(bottomVideoPath, bottomMovName + bottomLabelInfo + '.csv')

In [4]:
importlib.reload(sys.modules['reaching_utils'])
importlib.reload(sys.modules['cv_utils'])

<module 'cv_utils' from 'D:\\python_code\\cv_utils.py'>

In [5]:
ru.makeReachingMovieLabeled(topMovPath, topLabelPath, bottomMovPath, bottomLabelPath, moviePath)

Save to: D:\reachingvideos\waterdrop\labeled\20200221_WT1_D1_top_labeled.avi
bottomMovPath: D:\DeepLabCut\projects\ReachingBottom-Sarah-2020-02-25\videos\20200221_WT1_D1_bottom.avi
fps: 60.0
['leftEar', 'rightEar', 'leftEye', 'rightEye', 'nose', 'leftWrist', 'rightWrist', 'waterDrop']
['nose', 'leftFrontPaw', 'rightFrontPaw', 'leftWrist', 'rightWrist', 'leftElbow', 'rightElbow', 'leftHindPaw', 'rightHindPaw', 'waterDrop', 'leftFist', 'rightFist', 'windowTL', 'windowTR', 'windowBL', 'windowBR']
{'leftEar': (255, 0, 0), 'rightEar': (255, 215, 0), 'leftEye': (80, 255, 0), 'rightEye': (0, 255, 135), 'nose': (0, 159, 255), 'leftWrist': (56, 0, 255), 'rightWrist': (255, 0, 239), 'waterDrop': (255, 0, 24)}
[INFO] cleaning up...


In [26]:
ru.getLabelDict(topLabelPath)
#np.linspace(0, 1, 5)

['leftEar', 'rightEar', 'leftEye', 'rightEye', 'nose', 'leftWrist', 'rightWrist', 'waterDrop']


{'leftEar': {'x': [941.290693283081,
   1192.1278009414673,
   1182.634940624237,
   1170.8503090143204,
   1159.1928308010101,
   1155.801902770996,
   1150.1182217597961,
   1150.092601776123,
   1141.750916838646,
   1128.4213695526123,
   1113.3475189208984,
   1101.057571053505,
   1085.4465399980545,
   1075.7307480871677,
   1058.475466132164,
   1049.135090827942,
   1039.9772815704346,
   1019.9666919112206,
   1011.2826542854309,
   1000.3293986320496,
   991.8372139930725,
   989.5732473134995,
   986.6495985984802,
   983.8463859558105,
   981.1196328401566,
   971.6347889900208,
   942.6549043655396,
   923.7886447012424,
   913.4432551860809,
   897.3511848449707,
   881.4398283958435,
   865.016441822052,
   858.6679768562317,
   858.6104803085327,
   850.3833972215652,
   846.139105796814,
   832.7009198665619,
   827.4448337554932,
   819.1875976920128,
   814.6127271652222,
   807.7744421958923,
   792.419312953949,
   789.6570343971252,
   794.1789107322693,
   794.8

In [ ]:
tuple(np.round(np.multiply(255, (0.267004, 0.004874, 0.329415, 1.0))))

In [44]:
viridis = cm.get_cmap('viridis', 12)
viridis(np.linspace(0, 1, 12))

array([[0.267004, 0.004874, 0.329415, 1.      ],
       [0.283072, 0.130895, 0.449241, 1.      ],
       [0.262138, 0.242286, 0.520837, 1.      ],
       [0.220057, 0.343307, 0.549413, 1.      ],
       [0.177423, 0.437527, 0.557565, 1.      ],
       [0.143343, 0.522773, 0.556295, 1.      ],
       [0.119512, 0.607464, 0.540218, 1.      ],
       [0.166383, 0.690856, 0.496502, 1.      ],
       [0.319809, 0.770914, 0.411152, 1.      ],
       [0.525776, 0.833491, 0.288127, 1.      ],
       [0.762373, 0.876424, 0.137064, 1.      ],
       [0.993248, 0.906157, 0.143936, 1.      ]])

In [23]:
[x for x in range(1,10,3)]

[1, 4, 7]

In [37]:
jf = os.path.join(dDrive, 'academy_data', 'HoldBiteData_M18_M19_M20_M22', 'Analysis', 'MovieInfo', 'MovieInfo_M18_HoldBite_1559464469.json')
updateMovieInfoFile(jf, 'FrontMovie', (180, 100, 500, 340), key2='Window')
updateMovieInfoFile(jf, 'SideMovie', (120, 150, 440, 390), key2='Window')
movInfo = hbu.loadData(jf)

In [34]:
movInfoJsons

array(['MovieInfo_M23_HoldBite_1571203911.json',
       'MovieInfo_M23_HoldBite_1571205553.json',
       'MovieInfo_M23_HoldBite_1571206833.json',
       'MovieInfo_M23_HoldBite_1571208085.json',
       'MovieInfo_M23_HoldBite_1571209413.json',
       'MovieInfo_M23_HoldBite_1571291468.json',
       'MovieInfo_M23_HoldBite_1571292785.json',
       'MovieInfo_M23_HoldBite_1571293787.json',
       'MovieInfo_M23_HoldBite_1571383479.json',
       'MovieInfo_M23_HoldBite_1571384644.json',
       'MovieInfo_M23_HoldBite_1571440952.json',
       'MovieInfo_M23_HoldBite_1571443066.json',
       'MovieInfo_M23_HoldBite_1571449705.json',
       'MovieInfo_M23_HoldBite_1571554888.json',
       'MovieInfo_M23_HoldBite_1571558458.json',
       'MovieInfo_M23_HoldBite_1571559146.json',
       'MovieInfo_M23_HoldBite_1571560679.json',
       'MovieInfo_M23_HoldBite_1571571084.json',
       'MovieInfo_M23_HoldBite_1571609526.json',
       'MovieInfo_M23_HoldBite_1571611435.json',
       'MovieInfo_M2

In [12]:

movInfo = loadData(os.path.join(movInfoPath, movInfoJsons[32]))
fcamFolder, movTimeStr = os.path.split(movInfo['FrontMovie']['Folder'])
fImPaths = [os.path.join(fcamFolder, movTimeStr, imName) for imName in movInfo['FrontMovie']['ImageNames']]
sessionPath, sessionFile = os.path.split(movInfo['SessionFile'])
sessionName, jext = os.path.splitext(sessionFile)
saveToPath = os.path.join(fcamFolder, sessionName+'_FrontVideo.avi')
makeSingleVideo(fImPaths, movInfo['FrontMovie']['Timestamps'], saveToPath, window=(180, 110, 500, 350), rotate=True, codec='MJPG',ext='.avi')


#makeHoldBiteMovie(movInfo, saveToFolder)

hf: 240
wf: 320
FrameRate: 30
[INFO] cleaning up...


In [15]:
scamFolder, movTimeStr = os.path.split(movInfo['SideMovie']['Folder'])
sImPaths = [os.path.join(scamFolder, movTimeStr, imName) for imName in movInfo['SideMovie']['ImageNames']]
saveToSPath = os.path.join(scamFolder, sessionName+'_SideVideo.avi')
makeSingleVideo(sImPaths, movInfo['SideMovie']['Timestamps'], saveToSPath, window=(300, 120, 620, 360), rotate=True, codec='MJPG',ext='.avi')


hf: 240
wf: 320
FrameRate: 30
[INFO] cleaning up...


In [61]:
makeSingleVideo(fImPaths, movInfo['FrontMovie']['Timestamps'], saveToPath, window=(180, 110, 500, 350), rotate=True, codec='MJPG',ext='.avi')


FrameRate: 29


KeyboardInterrupt: 

In [16]:

makeSingleVideo(frontImPaths, frameTimes, saveToPath, window=(180, 110, 500, 350), rotate=True, codec='MJPG',ext='.avi')
makeSingleVideo(sideImPaths, frameTimes, saveToPath, window=(300, 120, 620, 360), rotate=True, codec='MJPG',ext='.avi')


1.0

In [334]:
cv2.destroyAllWindows()